## IBM Applied Data Science Capstone Course by Coursera
### Week 5 Final Report - Opening a Gym in Dubai, United Arab Emirates

1. Build a dataframe of communities in Dubai, UAE by aquiring the data from the Wikipedia page.
2. Get the geographical coordinates of the communities.
3. Obtain the venue data for each community using Foursquare
4. Create clusers from the venues data
5. Select the best place to open a new Gym

1. Import libraries

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge geocoder --yes 
import geocoder # to get coordinates

!conda install -c conda-forge BeautifulSoup4 --yes 
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print("Libraries imported.")

2. Scrap data from Wikipedia page into a DataFrame

In [7]:
# send the GET request and parse data from the wiki into a beautifulsoup object
data = requests.get("https://en.wikipedia.org/wiki/Category:Communities_in_Dubai").text
soup = BeautifulSoup(data, 'html.parser')

In [8]:
# create a list to store Communities data
Communities = []

In [10]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    Communities.append(row.text)

In [29]:
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Communities": Communities})
kl_df = kl_df.iloc[1:]
kl_df.head()

,Communities
1,Abu Hail
2,Al Amardhi
3,Al Bada
4,Al Baraha
5,Al Barsha


3. Get the geographical coordinates

In [30]:
# define the function to get coordinates of the communities
def get_latlng(Communities):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Dubai, ARE'.format(Communities))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [31]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(Communities) for Communities in kl_df["Communities"].tolist() ]

In [36]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']
kl_df.drop(kl_df.tail(1).index,inplace=True)
# check the Communities and the coordinates
kl_df

,Communities,Latitude,Longitude
1,Abu Hail,25.198395,55.52126
2,Al Amardhi,25.218610,55.26406
3,Al Bada,25.282800,55.31678
4,Al Baraha,25.105640,55.20057
5,Al Barsha,25.264060,55.29994
6,Al Bastakiya,25.269250,55.29944
7,Al Buteen,25.272170,55.30157
8,Al Dhagaya,25.276070,55.31087
9,Al Furjan,25.243370,55.35267
10,Al Garhoud,25.298710,55.33546


4. Create a map of Dubai with Communities superimposed on top

In [37]:
address = 'Dubai, ARE'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [38]:
# create map of Dubai using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, Communities in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Communities']):
    label = '{}'.format(Communities)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

5. Use the Foursquare API to explore the Communities

In [39]:
# define Foursquare Credentials and Version
CLIENT_ID = '4QNMNO1OHPWZXOWTKTZK3JLSBSOT53J2FQRVYK214UHBKJZG' 
CLIENT_SECRET = '5VMAKEMS03DXNVR1FEUJ0VDBYJAKS3Q3GFVYANQAXRUDAN5J'
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4QNMNO1OHPWZXOWTKTZK3JLSBSOT53J2FQRVYK214UHBKJZG
CLIENT_SECRET:5VMAKEMS03DXNVR1FEUJ0VDBYJAKS3Q3GFVYANQAXRUDAN5J


In [41]:
#get the top 100 venues that are within a radius of 2000 meters.
radius = 1500
LIMIT = 100

venues = []

for lat, long, Communities in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Communities']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            Communities,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))
        
        

In [42]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Communities', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(5946, 7)


,Communities,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Abu Hail,25.198395,55.52126,Seekh Kabab Restaurant مطعم سيخ كباب,25.200201,55.519778,Kebab Restaurant
1,Abu Hail,25.198395,55.52126,مزرعة البسام Al Bassam Farm,25.207041,55.515615,Farm
2,Abu Hail,25.198395,55.52126,Al Hamra Stables,25.208167,55.525684,Stables
3,Abu Hail,25.198395,55.52126,Wahat AlSahraa Nurseries (مشتل واحة الصحراء),25.194601,55.508499,Garden
4,Abu Hail,25.198395,55.52126,Desert Group,25.194296,55.508456,Garden Center


In [43]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Kebab Restaurant', 'Farm', 'Stables', 'Garden', 'Garden Center',
       'Restaurant', 'Gym / Fitness Center', 'Café', 'Shopping Plaza',
       'Coffee Shop', 'Mediterranean Restaurant', 'Shopping Mall',
       'Gluten-free Restaurant', 'Plaza', 'Cosmetics Shop', 'Beach',
       'Multiplex', 'Ice Cream Shop', 'Indoor Play Area', 'Burger Joint',
       'Hotel', 'Turkish Restaurant', 'Movie Theater', 'Dessert Shop',
       'Comfort Food Restaurant', 'Resort', "Men's Store", 'Gift Shop',
       'Park', 'Breakfast Spot', 'Lounge', 'Track Stadium', 'Pharmacy',
       'Lebanese Restaurant', 'Fast Food Restaurant', 'Sushi Restaurant',
       'Vegetarian / Vegan Restaurant', 'Arcade', 'Toy / Game Store',
       'American Restaurant', 'Botanical Garden', 'Indian Restaurant',
       'Chinese Restaurant', 'Portuguese Restaurant', 'Steakhouse', 'Bar',
       'Pool', 'Japanese Restaurant', 'Middle Eastern Restaurant',
       'Pilates Studio'], dtype=object)

In [45]:
# check if the results contain "Gym"
"Gym" in venues_df['VenueCategory'].unique()

True

6. Analyze Each Community

In [46]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add community column back to dataframe
kl_onehot['Communities'] = venues_df['Communities'] 

# move community column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(5946, 301)


,Communities,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Belgian Restaurant,Big Box Store,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Chinese Restaurant,Chocolate Shop,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cruise,Cultural Center,Cupcake Shop,Currency Exchange,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hong Kong Restaurant,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Iraqi Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kurdish Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Lighting Store,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mobility Store,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Public Art,Racetrack,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shawarma Place,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Syrian Restaurant,Tailor Shop,Tapas Res

In [47]:
#let's group rows by community and by taking the mean of the frequency of occurrence of each category

kl_grouped = kl_onehot.groupby(["Communities"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(78, 301)


,Communities,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Belgian Restaurant,Big Box Store,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Chinese Restaurant,Chocolate Shop,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cruise,Cultural Center,Cupcake Shop,Currency Exchange,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hong Kong Restaurant,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Iraqi Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kurdish Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Lighting Store,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mobility Store,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Public Art,Racetrack,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shawarma Place,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Syrian Restaurant,Tailor Shop,Tapas Res

In [48]:
# Create a new DataFrame for Gym data only
kl_mall = kl_grouped[["Communities","Gym"]]
kl_mall.head()

,Communities,Gym
0,Abu Hail,0.000000
1,Al Amardhi,0.000000
2,Al Bada,0.013333
3,Al Baraha,0.000000
4,Al Barsha,0.010000


7. Cluster Communities

In [49]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Communities"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 2, 0], dtype=int32)

In [50]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each community.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [51]:
kl_merged.rename(columns={"Communities": "Communities"}, inplace=True)
kl_merged.head()

,Communities,Gym,Cluster Labels
0,Abu Hail,0.000000,0
1,Al Amardhi,0.000000,0
2,Al Bada,0.013333,0
3,Al Baraha,0.000000,0
4,Al Barsha,0.010000,0


In [52]:
# merge kl_merged with kl_df to add latitude/longitude for each community
kl_merged = kl_merged.join(kl_df.set_index("Communities"), on="Communities")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(78, 5)


,Communities,Gym,Cluster Labels,Latitude,Longitude
0,Abu Hail,0.000000,0,25.198395,55.52126
1,Al Amardhi,0.000000,0,25.218610,55.26406
2,Al Bada,0.013333,0,25.282800,55.31678
3,Al Baraha,0.000000,0,25.105640,55.20057
4,Al Barsha,0.010000,0,25.264060,55.29994


In [53]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(78, 5)


,Communities,Gym,Cluster Labels,Latitude,Longitude
0,Abu Hail,0.000000,0,25.198395,55.52126
53,Emirates Hills,0.022727,0,25.277430,55.33746
52,Dubai Motor City,0.000000,0,25.069530,55.16620
50,"Deira, Dubai",0.010000,0,25.088910,55.14436
49,Culture Village,0.010000,0,25.277100,55.31062
48,Business Bay,0.013889,0,25.226700,55.34185
47,Bur Dubai,0.020000,0,25.187970,55.26273
54,Hor Al Anz,0.021739,0,25.135530,55.22238
46,Bu Kadra,0.010000,0,25.260400,55.28522
44,Arabian Ranches,0.010000,0,25.272550,55.30313


In [54]:
# Finally, let's visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Communities'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [55]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

In [56]:
#Cluster 1
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Communities,Gym,Cluster Labels,Latitude,Longitude
0,Abu Hail,0.000000,0,25.198395,55.52126
53,Emirates Hills,0.022727,0,25.277430,55.33746
52,Dubai Motor City,0.000000,0,25.069530,55.16620
50,"Deira, Dubai",0.010000,0,25.088910,55.14436
49,Culture Village,0.010000,0,25.277100,55.31062
48,Business Bay,0.013889,0,25.226700,55.34185
47,Bur Dubai,0.020000,0,25.187970,55.26273
54,Hor Al Anz,0.021739,0,25.135530,55.22238
46,Bu Kadra,0.010000,0,25.260400,55.28522
44,Arabian Ranches,0.010000,0,25.272550,55.30313


In [57]:
#Cluster 2
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Communities,Gym,Cluster Labels,Latitude,Longitude
60,Mohammed bin Rashid City,0.125000,1,25.25765,55.41692
62,Nad Al Hammar,0.142857,1,25.14436,55.35040


In [58]:
# Cluster 3
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Communities,Gym,Cluster Labels,Latitude,Longitude
12,Al Hudaiba,0.034483,2,25.22054,55.34166
8,Al Furjan,0.030000,2,25.24337,55.35267
16,Al Kefaf,0.030000,2,25.27177,55.33762
57,Kempinski Residences Palm Jumeirah,0.043478,2,25.22614,55.42421
21,Al Markada,0.047619,2,25.24506,55.43979
40,Al Twar,0.048780,2,25.29173,55.33822
55,Jumeirah,0.050000,2,25.07306,55.14367
76,Umm Suqeim,0.031915,2,25.16744,55.40708
18,Al Mamzar,0.030000,2,25.14579,55.20958
51,Dubai Marina,0.074074,2,25.04643,55.23820


### Conclusion
Most of the gyms are concentrated in the newly built areas in Dubai, with the highest number in cluster 2 and moderate number in cluster 3. On the other hand, cluster 1 has very low number to totally no Gyms in the community. This represents a great opportunity and high potential areas to open new gyms as there is very little to no competition from existing gyms. Meanwhile, gyms in cluster 2 are likely suffering from intense competition due to oversupply and high concentration of other gyms. From another perspective, this also shows that the oversupply of gyms mostly happened in the newly built areas of the city, with the suburb area still have very few gyms. Therefore, this project recommends property developers or investors to capitalize on these findings to open a new gym in communities in cluster 1 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new gyms in Communities in cluster 3 with moderate competition. Lastly, property developers are advised to avoid Communities in cluster 2 which already have high concentration of gyms and suffering from intense competition.